In [1]:
import boto3

client=boto3.client('sts')
account=client.get_caller_identity()['Account']

my_session=boto3.session.Session()
region=my_session.region_name

algorithm_name="pytorch-tft-container-test"
ecr_image='{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account, region, algorithm_name)

ecr_image

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


'551329315830.dkr.ecr.us-east-1.amazonaws.com/pytorch-tft-container-test:latest'

In [2]:
! aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-east-1.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [3]:
import sagemaker
import uuid

sagemaker_session = sagemaker.Session()
print('SageMaker version: ' + sagemaker.__version__)

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/DEMO-pytorch-cnn-cifar10'

role = sagemaker.get_execution_role()
checkpoint_suffix = str(uuid.uuid4())[:8]
checkpoint_s3_path = 's3://{}/checkpoints/checkpoint-{}'.format(bucket, checkpoint_suffix)

print('Checkpointing Path: {}'.format(checkpoint_s3_path))

SageMaker version: 2.103.0
Checkpointing Path: s3://sagemaker-us-east-1-551329315830/checkpoints/checkpoint-bdc3a2df


In [4]:
from utils_timeseries import download_process_and_return_raw_data, save_local_and_upload_s3, metadata_json_upload_s3
import sagemaker
import uuid

max_prediction_length = 6
max_encoder_length = 24
special_days = [
        "easter_day",
        "good_friday",
        "new_year",
        "christmas",
        "labor_day",
        "independence_day",
        "revolution_day_memorial",
        "regional_games",
        "fifa_u_17_world_cup",
        "football_gold_cup",
        "beer_capital",
        "music_fest",
    ]

training_metadata = {}
training_metadata['time_idx'] = "time_idx"
training_metadata['target'] = "volume"
training_metadata['group_ids'] = ["agency", "sku"]
training_metadata['min_encoder_length'] = max_encoder_length // 2      # keep encoder length long (as it is in the validation set)
training_metadata['max_encoder_length'] = max_encoder_length
training_metadata['min_prediction_length'] = 1      
training_metadata['max_prediction_length'] = max_prediction_length
training_metadata['static_categoricals'] = ["agency", "sku"]
training_metadata['static_reals'] = ["avg_population_2017", "avg_yearly_household_income_2017"]
training_metadata['time_varying_known_categoricals'] = ["special_days", "month"]
training_metadata['variable_groups'] = {"special_days": special_days}
training_metadata['time_varying_known_reals'] = ["time_idx", "price_regular", "discount_in_percent"]
training_metadata['time_varying_unknown_categoricals'] = []
training_metadata['time_varying_unknown_reals'] = [
        "volume",
        "log_volume",
        "industry_volume",
        "soda_volume",
        "avg_max_temp",
        "avg_volume_by_agency",
        "avg_volume_by_sku",
    ]
training_metadata['target_normalizer'] = {
                            "normalized_groups": ["agency", "sku"],
                            "normalization_transformation": 'softplus'
                        }
training_metadata['add_relative_time_idx'] = True
training_metadata['add_target_scales'] = True
training_metadata['add_encoder_length'] = True




sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

# obtain & upload training data
training_data = download_process_and_return_raw_data()
inputs = save_local_and_upload_s3(training_data, sagemaker_session, bucket, data_filename="stallion_data")

# upload metadata
training_metadata['training_cutoff'] = int(training_data["time_idx"].max() - max_prediction_length)
metadata_json_upload_s3(training_metadata, sagemaker_session, bucket, metadata_filename="stallion_metadata")

Checkpointing directory timeseries_data exists
saved raw data to timeseries_data/stallion_data.parquet
Checkpointing directory timeseries_data exists
saved metadata to timeseries_data/stallion_metadata.json


's3://sagemaker-us-east-1-551329315830/data/timeseries_data'

In [6]:
use_spot_instances = False
max_run=6000      # in seconds, after this, job will be terminated
max_wait = 10 * max_run if use_spot_instances else None
local_image_name = 'pytorch-tft-container-test'

In [7]:
from sagemaker.pytorch import PyTorch
from sagemaker.estimator import Estimator
from sagemaker.debugger import TensorBoardOutputConfig


hyperparameters = {
        'epochs': 100,
        'data-filename': "stallion_data.parquet",
        'metadata-filename': "stallion_metadata.json"
    }

tensorboard_output_config = TensorBoardOutputConfig(
    s3_output_path='s3://sagemaker-us-east-1-551329315830/tensorboard',
    container_local_output_path='/lightning_logs'
)

spot_estimator  = PyTorch(entry_point='TFT_docker/TFT.py',
                            dependencies=['TFT_docker/requirements.txt'],
                            role=role,
                            framework_version='1.7.1',
                            py_version='py3',
                            instance_count=1,
#                             instance_type='local',
#                             instance_type='ml.p3.2xlarge',
                            instance_type='ml.p2.xlarge',
                            base_job_name='tft-pytorch-spot-1',
                            hyperparameters=hyperparameters,
                            checkpoint_s3_uri=checkpoint_s3_path,
                            debugger_hook_config=False,
                            input_mode = 'FastFile',
                            use_spot_instances=use_spot_instances,
                            max_run=max_run,
                            max_wait=max_wait,
                            tensorboard_output_config=tensorboard_output_config
                           )

spot_estimator.fit(
                inputs,
                logs = 'All'
            )

2022-08-22 10:05:33 Starting - Starting the training job...ProfilerReport-1661162732: InProgress
...
2022-08-22 10:06:20 Starting - Preparing the instances for training......
2022-08-22 10:07:31 Downloading - Downloading input data......
2022-08-22 10:08:28 Training - Downloading the training image...........................
2022-08-22 10:12:49 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-08-22 10:12:52,570 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-08-22 10:12:52,599 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-08-22 10:12:52,611 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-08-22 10:12:53,139 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.6 -m pip instal

  Created wheel for fastparquet: filename=fastparquet-0.8.0-cp36-cp36m-linux_x86_64.whl size=1256936 sha256=acce5b84cafd8efc1d34084d35f17fa2a3b1d53d99d17566df9ff57f7f4dce0d
  Stored in directory: /root/.cache/pip/wheels/6a/6a/4f/0fd8e8bcbc4b5b751186e363b5b03975d8643eee2975eed2ca
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-py3-none-any.whl size=3161 sha256=230e21aaaed3193563de44012de2bf889d9101cb3a18d3e870d154b49f72ed85
  Stored in directory: /root/.cache/pip/wheels/6a/f5/9c/f8331a854f7a8739cf0e74c13854e4dd7b1af11b04fe1dde13
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11106 sha256=aecad7c1a4797222c5df3c2e473a1af49ac9dab8751b583b7a930629ea7f51dd
  Stored in directory: /root/.cache/pip/wheels/95/38/95/e30a7f0b44cb90642de3469f211a3218f93f871789b4f4b46c
Successfully built fastparquet idna-ssl pyperclip
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.0.1
    Uninstalling importlib-metadata-4.0.1:
   

Distributed training - False
Checkpointing directory /opt/ml/checkpoints exists
Device Type: cuda
Load Time Series dataset from S3
creating dataloader
get GPU information
GPU count: 1
create model trainer
create model from dataset
Number of parameters in network: 29.7k
training model
#015Validation sanity check: 0it [00:00, ?it/s]#015Validation sanity check:   0%|          | 0/1 [00:00<?, ?it/s]#015Validation sanity check: 100%|██████████| 1/1 [00:03<00:00,  3.36s/it]#015                                                                      #015#015Training: 0it [00:00, ?it/s]#015Training:   0%|          | 0/31 [00:00<?, ?it/s]#015Epoch 0:   0%|          | 0/31 [00:00<?, ?it/s] #015Epoch 0:   3%|▎         | 1/31 [00:01<00:41,  1.38s/it]#015Epoch 0:   3%|▎         | 1/31 [00:01<00:41,  1.38s/it, loss=135, v_num=0, train_loss_step=135.0]#015Epoch 0:   6%|▋         | 2/31 [00:01<00:26,  1.08it/s, loss=135, v_num=0, train_loss_step=135.0]#015Epoch 0:   6%|▋         | 2/31 [00:01<00:26,  1.0

#015                                                         #033[A#015Epoch 1: 100%|██████████| 31/31 [00:20<00:00,  1.54it/s, loss=73.9, v_num=0, train_loss_step=71.70, val_loss=104.0, train_loss_epoch=73.50]#015Epoch 1:   0%|          | 0/31 [00:00<?, ?it/s, loss=73.9, v_num=0, train_loss_step=71.70, val_loss=104.0, train_loss_epoch=73.50]         #015Epoch 2:   0%|          | 0/31 [00:00<?, ?it/s, loss=73.9, v_num=0, train_loss_step=71.70, val_loss=104.0, train_loss_epoch=73.50]#015Epoch 2:   3%|▎         | 1/31 [00:01<00:32,  1.07s/it, loss=73.9, v_num=0, train_loss_step=71.70, val_loss=104.0, train_loss_epoch=73.50]#015Epoch 2:   3%|▎         | 1/31 [00:01<00:32,  1.07s/it, loss=74.8, v_num=0, train_loss_step=83.00, val_loss=104.0, train_loss_epoch=73.50]#015Epoch 2:   6%|▋         | 2/31 [00:01<00:22,  1.29it/s, loss=74.8, v_num=0, train_loss_step=83.00, val_loss=104.0, train_loss_epoch=73.50]#015Epoch 2:   6%|▋         | 2/31 [00:01<00:22,  1.29it/s, loss=74.3, v_num=0, train_l

#015                                                         #033[A#015Epoch 2: 100%|██████████| 31/31 [00:20<00:00,  1.53it/s, loss=63, v_num=0, train_loss_step=63.60, val_loss=99.00, train_loss_epoch=65.10]#015Epoch 2:   0%|          | 0/31 [00:00<?, ?it/s, loss=63, v_num=0, train_loss_step=63.60, val_loss=99.00, train_loss_epoch=65.10]         #015Epoch 3:   0%|          | 0/31 [00:00<?, ?it/s, loss=63, v_num=0, train_loss_step=63.60, val_loss=99.00, train_loss_epoch=65.10]#015Epoch 3:   3%|▎         | 1/31 [00:01<00:30,  1.03s/it, loss=63, v_num=0, train_loss_step=63.60, val_loss=99.00, train_loss_epoch=65.10]#015Epoch 3:   3%|▎         | 1/31 [00:01<00:30,  1.03s/it, loss=61.9, v_num=0, train_loss_step=60.70, val_loss=99.00, train_loss_epoch=65.10]#015Epoch 3:   6%|▋         | 2/31 [00:01<00:22,  1.29it/s, loss=61.9, v_num=0, train_loss_step=60.70, val_loss=99.00, train_loss_epoch=65.10]#015Epoch 3:   6%|▋         | 2/31 [00:01<00:22,  1.29it/s, loss=61.4, v_num=0, train_loss_step

...

#015                                                         #033[A#015Epoch 18: 100%|██████████| 31/31 [00:19<00:00,  1.55it/s, loss=44.8, v_num=0, train_loss_step=49.20, val_loss=77.40, train_loss_epoch=46.70]#015Epoch 18:   0%|          | 0/31 [00:00<?, ?it/s, loss=44.8, v_num=0, train_loss_step=49.20, val_loss=77.40, train_loss_epoch=46.70]         #015Epoch 19:   0%|          | 0/31 [00:00<?, ?it/s, loss=44.8, v_num=0, train_loss_step=49.20, val_loss=77.40, train_loss_epoch=46.70]#015Epoch 19:   3%|▎         | 1/31 [00:01<00:31,  1.05s/it, loss=44.8, v_num=0, train_loss_step=49.20, val_loss=77.40, train_loss_epoch=46.70]#015Epoch 19:   3%|▎         | 1/31 [00:01<00:31,  1.05s/it, loss=45.4, v_num=0, train_loss_step=48.70, val_loss=77.40, train_loss_epoch=46.70]#015Epoch 19:   6%|▋         | 2/31 [00:01<00:23,  1.24it/s, loss=45.4, v_num=0, train_loss_step=48.70, val_loss=77.40, train_loss_epoch=46.70]#015Epoch 19:   6%|▋         | 2/31 [00:01<00:23,  1.24it/s, loss=45, v_num=0, tr

#015                                                         #033[A#015Epoch 19: 100%|██████████| 31/31 [00:20<00:00,  1.51it/s, loss=47.2, v_num=0, train_loss_step=49.40, val_loss=77.40, train_loss_epoch=47.20]#015Epoch 19:   0%|          | 0/31 [00:00<?, ?it/s, loss=47.2, v_num=0, train_loss_step=49.40, val_loss=77.40, train_loss_epoch=47.20]         #015Epoch 20:   0%|          | 0/31 [00:00<?, ?it/s, loss=47.2, v_num=0, train_loss_step=49.40, val_loss=77.40, train_loss_epoch=47.20]#015Epoch 20:   3%|▎         | 1/31 [00:01<00:33,  1.12s/it, loss=47.2, v_num=0, train_loss_step=49.40, val_loss=77.40, train_loss_epoch=47.20]#015Epoch 20:   3%|▎         | 1/31 [00:01<00:33,  1.12s/it, loss=47.7, v_num=0, train_loss_step=47.60, val_loss=77.40, train_loss_epoch=47.20]#015Epoch 20:   6%|▋         | 2/31 [00:01<00:24,  1.19it/s, loss=47.7, v_num=0, train_loss_step=47.60, val_loss=77.40, train_loss_epoch=47.20]#015Epoch 20:   6%|▋         | 2/31 [00:01<00:24,  1.19it/s, loss=47.7, v_num=0, 

#015                                                         #033[A#015Epoch 20: 100%|██████████| 31/31 [00:20<00:00,  1.50it/s, loss=49.1, v_num=0, train_loss_step=41.40, val_loss=77.70, train_loss_epoch=48.70]#015Epoch 20:   0%|          | 0/31 [00:00<?, ?it/s, loss=49.1, v_num=0, train_loss_step=41.40, val_loss=77.70, train_loss_epoch=48.70]         #015Epoch 21:   0%|          | 0/31 [00:00<?, ?it/s, loss=49.1, v_num=0, train_loss_step=41.40, val_loss=77.70, train_loss_epoch=48.70]#015Epoch 21:   3%|▎         | 1/31 [00:01<00:31,  1.06s/it, loss=49.1, v_num=0, train_loss_step=41.40, val_loss=77.70, train_loss_epoch=48.70]#015Epoch 21:   3%|▎         | 1/31 [00:01<00:31,  1.06s/it, loss=48.9, v_num=0, train_loss_step=45.50, val_loss=77.70, train_loss_epoch=48.70]#015Epoch 21:   6%|▋         | 2/31 [00:01<00:23,  1.21it/s, loss=48.9, v_num=0, train_loss_step=45.50, val_loss=77.70, train_loss_epoch=48.70]#015Epoch 21:   6%|▋         | 2/31 [00:01<00:23,  1.21it/s, loss=49.1, v_num=0, 

#015                                                         #033[A#015Epoch 21: 100%|██████████| 31/31 [00:21<00:00,  1.46it/s, loss=47.6, v_num=0, train_loss_step=40.80, val_loss=76.40, train_loss_epoch=47.60]#015Epoch 21:   0%|          | 0/31 [00:00<?, ?it/s, loss=47.6, v_num=0, train_loss_step=40.80, val_loss=76.40, train_loss_epoch=47.60]         #015Epoch 22:   0%|          | 0/31 [00:00<?, ?it/s, loss=47.6, v_num=0, train_loss_step=40.80, val_loss=76.40, train_loss_epoch=47.60]#015Epoch 22:   3%|▎         | 1/31 [00:01<00:35,  1.20s/it, loss=47.6, v_num=0, train_loss_step=40.80, val_loss=76.40, train_loss_epoch=47.60]#015Epoch 22:   3%|▎         | 1/31 [00:01<00:35,  1.20s/it, loss=48.4, v_num=0, train_loss_step=50.00, val_loss=76.40, train_loss_epoch=47.60]#015Epoch 22:   6%|▋         | 2/31 [00:01<00:24,  1.17it/s, loss=48.4, v_num=0, train_loss_step=50.00, val_loss=76.40, train_loss_epoch=47.60]#015Epoch 22:   6%|▋         | 2/31 [00:01<00:24,  1.17it/s, loss=47.6, v_num=0, 


2022-08-22 10:22:12 Uploading - Uploading generated training modelGPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
   | Name                               | Type                            | Params
----------------------------------------------------------------------------------------
0  | loss                               | QuantileLoss                    | 0     
1  | logging_metrics                    | ModuleList                      | 0     
2  | input_embeddings                   | MultiEmbedding                  | 1.3 K 
3  | prescalers                         | ModuleDict                      | 256   
4  | static_variable_selection          | VariableSelectionNetwork        | 3.4 K 
5  | encoder_variable_selection         | VariableSelectionNetwork        | 8.0 K 
6  | decoder_variable_selection         | VariableSelectionNetwork        | 2.7 K 
7  | static_context_variable_

In [ ]:
spot_estimator.latest_job_tensorboard_artifacts_path()

In [ ]:
# # deploy the trained model
# predictor=estimator.deploy(1, instance_type)
tensorflow_logs_path = "lightning_logs"

In [ ]:
aws_region = sagemaker_session.boto_region_name
!AWS_REGION={aws_region}
!echo tensorboard --logdir {tensorflow_logs_path}

In [ ]:
!AWS_REGION=eu-east-1 tensorboard --logdir s3://sagemaker-us-east-1-551329315830/tensorboard/